In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('titanic.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [5]:
df.Age = df.Age.fillna(df.Age.mean())

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [6]:
df.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [7]:
df.Embarked = df.Embarked.fillna('S')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
df.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [10]:
df.Sex = df.Sex.apply(lambda x: 1 if x == 'male' else 0)

df.Sex.value_counts()

Sex
1    577
0    314
Name: count, dtype: int64

In [11]:
df = pd.get_dummies(df, columns=['Embarked'], dtype=int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [12]:
df.Survived.value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [13]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.Survived == 0]
df_minority = df[df.Survived == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement to allow for upsampling
                                 n_samples=len(df_majority),  # to match majority class size
                                 random_state=123) # reproducible results

# Combine upsampled minority class with majority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
df_balanced.Survived.value_counts()


Survived
0    549
1    549
Name: count, dtype: int64

In [14]:
from sklearn.preprocessing import MinMaxScaler

df = df_balanced.copy()

scaler = MinMaxScaler()

df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,0.268877,1,0,0.014151,0,0,1
4,0,3,1,0.432749,0,0,0.015713,0,0,1
5,0,3,1,0.365929,0,0,0.016510,0,1,0
6,0,1,1,0.672255,0,0,0.101229,0,0,1
7,0,3,1,0.016765,3,1,0.041136,0,0,1


In [15]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='Survived')
y = df.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.metrics import classification_report


from sklearn.svm import SVC

model = SVC()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Without hyperparameter tuning")
print(classification_report(y_test, y_pred))

Without hyperparameter tuning
              precision    recall  f1-score   support

           0       0.79      0.89      0.83       176
           1       0.85      0.73      0.78       154

    accuracy                           0.81       330
   macro avg       0.82      0.81      0.81       330
weighted avg       0.82      0.81      0.81       330



In [16]:
y_pred_train = model.predict(X_train)

print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.75      0.84      0.79       373
           1       0.83      0.73      0.78       395

    accuracy                           0.79       768
   macro avg       0.79      0.79      0.79       768
weighted avg       0.79      0.79      0.79       768



In [17]:
print(model.score(X_test, y_test))

0.8121212121212121


In [18]:
from sklearn.model_selection import GridSearchCV

model = SVC(max_iter=10000)

param_grid = [
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'degree': [2, 3, 4], 'coef0': [0.0, 0.1, 0.5, 1.0], 'kernel': ['poly']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'coef0': [0.0, 0.1, 0.5, 1.0], 'kernel': ['sigmoid']}
]

grid = GridSearchCV(model, param_grid, cv=5, verbose=3)

grid.fit(X_train, y_train)

print(grid.best_params_)

print(grid.best_score_)

y_pred = grid.best_estimator_.predict(X_test)

print("With hyperparameter tuning")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 430 candidates, totalling 2150 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.747 total time=   0.0s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.760 total time=   0.0s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.721 total time=   0.0s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.784 total time=   0.0s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.647 total time=   0.0s
[CV 1/5] END ................C=1, kernel=linear;, score=0.747 total time=   0.0s
[CV 2/5] END ................C=1, kernel=linear;, score=0.760 total time=   0.0s
[CV 3/5] END ................C=1, kernel=linear;, score=0.721 total time=   0.0s
[CV 4/5] END ................C=1, kernel=linear;, score=0.797 total time=   0.0s
[CV 5/5] END ................C=1, kernel=linear;, score=0.647 total time=   0.0s
[CV 1/5] END ...............C=10, kernel=linear;, score=0.740 total time=   0.0s
[CV 2/5] END ...............C=10, kernel=line

C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftware

[CV 3/5] END ..............C=100, kernel=linear;, score=0.740 total time=   0.0s
[CV 4/5] END ..............C=100, kernel=linear;, score=0.810 total time=   0.0s
[CV 5/5] END ..............C=100, kernel=linear;, score=0.647 total time=   0.0s
[CV 1/5] END .............C=1000, kernel=linear;, score=0.604 total time=   0.0s
[CV 2/5] END .............C=1000, kernel=linear;, score=0.506 total time=   0.0s
[CV 3/5] END .............C=1000, kernel=linear;, score=0.539 total time=   0.0s


C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftware

[CV 4/5] END .............C=1000, kernel=linear;, score=0.431 total time=   0.0s
[CV 5/5] END .............C=1000, kernel=linear;, score=0.627 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.727 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.714 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.695 total time=   0.0s


...

[CV 5/5] END C=1000, coef0=1.0, degree=4, gamma=1, kernel=poly;, score=0.464 total time=   0.0s
[CV 1/5] END C=1000, coef0=1.0, degree=4, gamma=0.1, kernel=poly;, score=0.786 total time=   0.0s
[CV 2/5] END C=1000, coef0=1.0, degree=4, gamma=0.1, kernel=poly;, score=0.747 total time=   0.0s
[CV 3/5] END C=1000, coef0=1.0, degree=4, gamma=0.1, kernel=poly;, score=0.708 total time=   0.0s
[CV 4/5] END C=1000, coef0=1.0, degree=4, gamma=0.1, kernel=poly;, score=0.765 total time=   0.0s
[CV 5/5] END C=1000, coef0=1.0, degree=4, gamma=0.1, kernel=poly;, score=0.752 total time=   0.0s
[CV 1/5] END C=1000, coef0=1.0, degree=4, gamma=0.01, kernel=poly;, score=0.792 total time=   0.0s
[CV 2/5] END C=1000, coef0=1.0, degree=4, gamma=0.01, kernel=poly;, score=0.799 total time=   0.0s
[CV 3/5] END C=1000, coef0=1.0, degree=4, gamma=0.01, kernel=poly;, score=0.766 total time=   0.0s


C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\norbe\AppData\Local\Packages\PythonSoftware

[CV 4/5] END C=1000, coef0=1.0, degree=4, gamma=0.01, kernel=poly;, score=0.778 total time=   0.0s
[CV 5/5] END C=1000, coef0=1.0, degree=4, gamma=0.01, kernel=poly;, score=0.752 total time=   0.0s
[CV 1/5] END C=1000, coef0=1.0, degree=4, gamma=0.001, kernel=poly;, score=0.747 total time=   0.0s
[CV 2/5] END C=1000, coef0=1.0, degree=4, gamma=0.001, kernel=poly;, score=0.766 total time=   0.0s
[CV 3/5] END C=1000, coef0=1.0, degree=4, gamma=0.001, kernel=poly;, score=0.727 total time=   0.0s
[CV 4/5] END C=1000, coef0=1.0, degree=4, gamma=0.001, kernel=poly;, score=0.817 total time=   0.0s
[CV 5/5] END C=1000, coef0=1.0, degree=4, gamma=0.001, kernel=poly;, score=0.654 total time=   0.0s
[CV 1/5] END C=1000, coef0=1.0, degree=4, gamma=0.0001, kernel=poly;, score=0.747 total time=   0.0s
[CV 2/5] END C=1000, coef0=1.0, degree=4, gamma=0.0001, kernel=poly;, score=0.760 total time=   0.0s
[CV 3/5] END C=1000, coef0=1.0, degree=4, gamma=0.0001, kernel=poly;, score=0.721 total time=   0.0s

C:\Users\norbe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 1/5] END C=0.1, coef0=0.0, gamma=1, kernel=sigmoid;, score=0.513 total time=   0.0s
[CV 2/5] END C=0.1, coef0=0.0, gamma=1, kernel=sigmoid;, score=0.513 total time=   0.0s
[CV 3/5] END C=0.1, coef0=0.0, gamma=1, kernel=sigmoid;, score=0.513 total time=   0.0s
[CV 4/5] END C=0.1, coef0=0.0, gamma=1, kernel=sigmoid;, score=0.516 total time=   0.0s
[CV 5/5] END C=0.1, coef0=0.0, gamma=1, kernel=sigmoid;, score=0.523 total time=   0.0s
[CV 1/5] END C=0.1, coef0=0.0, gamma=0.1, kernel=sigmoid;, score=0.656 total time=   0.0s
[CV 2/5] END C=0.1, coef0=0.0, gamma=0.1, kernel=sigmoid;, score=0.630 total time=   0.0s
[CV 3/5] END C=0.1, coef0=0.0, gamma=0.1, kernel=sigmoid;, score=0.675 total time=   0.0s
[CV 4/5] END C=0.1, coef0=0.0, gamma=0.1, kernel=sigmoid;, score=0.686 total time=   0.0s
[CV 5/5] END C=0.1, coef0=0.0, gamma=0.1, kernel=sigmoid;, score=0.556 total time=   0.0s
[CV 1/5] END C=0.1, coef0=0.0, gamma=0.01, kernel=sigmoid;, score=0.513 total time=   0.0s
[CV 2/5] END C=0.1,

In [22]:
est = grid.best_estimator_

print(est.score(X_test, y_test))
print(est.score(X_train, y_train))

print(grid.best_params_)

0.8242424242424242
0.8541666666666666
{'C': 100, 'gamma': 1, 'kernel': 'rbf'}


Hyperparameters are very important, from the GridSearch above, we can see that it had a 1% increase from the default model, but seeing the different results of the GridSearch, we can see that with the wrong hyperparameters, the accuracies can go as low as 50%